In [1]:
import pyspark
try:
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

# 2015-02 2do Recuperatorio
 
Un telescopio registra automaticamente la luminosidad de distintas estrellas generando un RDD con registros de tipo (star_id, magnitude,spectral_type, timestamp). Queremos obtener un listado de estrellas que tienen el mismo tipo espectral e igual promedio de magnitud una vez redondeado el mismo a un decimal. El resultado debe ser una lista en donde cada elemento de la lista es una lista de ids de estrellas similares.

In [2]:
stars = [
    (1, 5, 1, 1),
    (2, 10, 1, 1),
    (3, 6, 1, 1),
    (4, 5.5, 2, 1),
    (1, 6, 1, 2),
    (2, 9, 1, 2),
    (3, 5, 1, 2),
    (1, 5.5, 1, 3),
    (2, 11, 1, 3),
    (3, 5.5, 1, 3)
]

In [3]:
data = sc.parallelize(stars);

In [7]:
rss1 = data.map(lambda x: (x[0], (x[2], x[1], 1)))\
    
rss1.collect()

[(1, (1, 5, 1)),
 (2, (1, 10, 1)),
 (3, (1, 6, 1)),
 (4, (2, 5.5, 1)),
 (1, (1, 6, 1)),
 (2, (1, 9, 1)),
 (3, (1, 5, 1)),
 (1, (1, 5.5, 1)),
 (2, (1, 11, 1)),
 (3, (1, 5.5, 1))]

In [9]:
rss1b = rss1.reduceByKey(lambda x, y: (x[0], x[1]+y[1], x[2]+y[2]))

rss1b.collect()

[(4, (2, 5.5, 1)), (1, (1, 16.5, 3)), (2, (1, 30, 3)), (3, (1, 16.5, 3))]

In [5]:
rss2 = rss1b.map(lambda x: ((x[1][0], x[1][1]/x[1][2]), x[0]))
rss2.collect()

[((2, 5.5), 4), ((1, 5.5), 1), ((1, 10.0), 2), ((1, 5.5), 3)]

In [6]:
rss3 = rss2.groupByKey()\
            .map(lambda x: (x[0], list(x[1])))

rss3.collect()

[((1, 5.5), [1, 3]), ((1, 10.0), [2]), ((2, 5.5), [4])]